# W_GL_BALANCE_F Fact Table Load
### GL Balance Fact - Incremental Update from Staging
**Package:** SILOS_SIL_GLBALANCEFACT  
**Source Table:** W_GL_BALANCE_FS (staging)  
**Target Table:** W_GL_BALANCE_F (fact)  
**ETL_PROC_WID:** 2625555  
**Load Pattern:** Incremental (Insert + Update) with Change Detection

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL,
  380 AS DATASOURCE_NUM_ID,
  '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID,
  2625555 AS ETL_PROC_WID,
  12345 AS EXECUTION_ID,
  30 AS PRUNE_DAYS,
  COALESCE(
    (SELECT MIN(LAST_MAX_DATE) 
     FROM workspace.oracle_edw.w_etl_load_dates
     WHERE PACKAGE_NAME = 'SILOS_SIL_GLBALANCEFACT'
       AND DATASOURCE_NUM_ID = 380
       AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'),
    to_timestamp('1970-01-01 00:00:00', 'yyyy-MM-dd HH:mm:ss')
  ) AS LAST_EXTRACT_DATE
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLBALANCEFACT'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1';

In [ ]:
%sql
-- 2) SOURCE STAGING: Read from W_GL_BALANCE_FS staging table
-- NOTE: Staging table already filtered incrementally during extraction
CREATE OR REPLACE TEMP VIEW stg_balance_source AS
SELECT
  fs.*,
  -- Create composite integration ID
  CONCAT(
    CAST(fs.DATASOURCE_NUM_ID AS STRING), '~',
    CAST(fs.LEDGER_ID AS STRING), '~',
    CAST(fs.GL_ACCOUNT_ID AS STRING), '~',
    CAST(fs.BALANCE_DATE AS STRING), '~',
    COALESCE(fs.BALANCE_TYPE_ID, '__NA__')
  ) AS INTEGRATION_ID
FROM workspace.oracle_edw.w_gl_balance_fs fs;

In [ ]:
%sql
-- 3) DIMENSION LOOKUPS: Join with all required dimensions
CREATE OR REPLACE TEMP VIEW stg_balance_with_dimensions AS
SELECT
  src.INTEGRATION_ID,
  src.DATASOURCE_NUM_ID,
  
  -- Core dimensions
  ledger.ROW_WID AS LEDGER_WID,
  acct.ROW_WID AS GL_ACCOUNT_WID,
  nat_acct.ROW_WID AS NATURAL_ACCOUNT_WID,
  bal_seg.ROW_WID AS BALANCING_SEGMENT_WID,
  cc.ROW_WID AS COST_CENTER_WID,
  co_org.ROW_WID AS COMPANY_ORG_WID,
  ba_org.ROW_WID AS BUSN_AREA_ORG_WID,
  
  -- Calendar/Time dimensions
  mcal.ROW_WID AS MCAL_CAL_WID,
  CAST(CONCAT(
    COALESCE(CAST(mcal.ROW_WID AS STRING), '0'),
    DATE_FORMAT(src.BALANCE_DATE, 'yyyyMMdd'),
    CASE WHEN src.ADJUSTMENT_FLAG = 'Y' THEN '999' ELSE '000' END
  ) AS BIGINT) AS BALANCE_DT_WID,
  
  CAST(DATE_FORMAT(src.BALANCE_DATE, 'HHmmss') AS BIGINT) AS BALANCE_TM_WID,
  
  -- Segment dimensions (showing first 5 as example)
  seg1.ROW_WID AS GL_SEGMENT1_WID,
  seg2.ROW_WID AS GL_SEGMENT2_WID,
  seg3.ROW_WID AS GL_SEGMENT3_WID,
  seg4.ROW_WID AS GL_SEGMENT4_WID,
  seg5.ROW_WID AS GL_SEGMENT5_WID,
  
  -- Other dimensions
  treas.ROW_WID AS TREASURY_SYMBOL_WID,
  dataset_sec.ROW_WID AS GL_DATASET_SEC_WID,
  balance_type.ROW_WID AS BALANCE_TYPE_WID,
  budget.ROW_WID AS BUDGET_WID,
  budget_period.ROW_WID AS BUDGET_PERIOD_WID,
  proj_seg.ROW_WID AS SEG_PROJECT_WID,
  prog_seg.ROW_WID AS SEG_PROGRAM_WID,
  
  -- Measures (amounts)
  src.BALANCE_ACCT_AMT,
  src.BALANCE_LOC_AMT,
  src.BALANCE_LOC_AMT * COALESCE(exch_rate.GLOBAL1_EXCH_RATE, 1) AS BALANCE_GLOBAL1_AMT,
  src.BALANCE_LOC_AMT * COALESCE(exch_rate.GLOBAL2_EXCH_RATE, 1) AS BALANCE_GLOBAL2_AMT,
  src.BALANCE_LOC_AMT * COALESCE(exch_rate.GLOBAL3_EXCH_RATE, 1) AS BALANCE_GLOBAL3_AMT,
  
  src.ACTIVITY_ACCT_AMT,
  src.ACTIVITY_LOC_AMT,
  src.ACTIVITY_LOC_AMT * COALESCE(exch_rate.GLOBAL1_EXCH_RATE, 1) AS ACTIVITY_GLOBAL1_AMT,
  src.ACTIVITY_LOC_AMT * COALESCE(exch_rate.GLOBAL2_EXCH_RATE, 1) AS ACTIVITY_GLOBAL2_AMT,
  src.ACTIVITY_LOC_AMT * COALESCE(exch_rate.GLOBAL3_EXCH_RATE, 1) AS ACTIVITY_GLOBAL3_AMT,
  
  src.CARRY_FORWARD_ACCT_AMT,
  src.CARRY_FORWARD_LOC_AMT,
  src.CARRY_FORWARD_LOC_AMT * COALESCE(exch_rate.GLOBAL1_EXCH_RATE, 1) AS CARRY_FORWARD_GLOBAL1_AMT,
  src.CARRY_FORWARD_LOC_AMT * COALESCE(exch_rate.GLOBAL2_EXCH_RATE, 1) AS CARRY_FORWARD_GLOBAL2_AMT,
  src.CARRY_FORWARD_LOC_AMT * COALESCE(exch_rate.GLOBAL3_EXCH_RATE, 1) AS CARRY_FORWARD_GLOBAL3_AMT,
  
  -- Attributes
  src.DB_CR_IND,
  src.ACCT_CURR_CODE,
  src.LOC_CURR_CODE,
  src.BALANCE_TYPE_FLAG,
  src.TRANSLATED_FLAG,
  src.SUMMARY_ACCOUNT_FLAG,
  src.FINANCIAL_GL_FLG,
  src.BUDGETARY_CONTROL_FLG,
  src.BALANCE_ID,
  
  -- Audit columns
  src.CREATED_ON_DT,
  src.CHANGED_ON_DT,
  src.AUX1_CHANGED_ON_DT,
  src.AUX2_CHANGED_ON_DT,
  src.AUX3_CHANGED_ON_DT,
  src.AUX4_CHANGED_ON_DT,
  
  -- Tenant/Custom
  src.TENANT_ID,
  src.X_CUSTOM
  
FROM stg_balance_source src

-- Ledger dimension
LEFT JOIN workspace.oracle_edw.w_ledger_d ledger
  ON src.DATASOURCE_NUM_ID = ledger.DATASOURCE_NUM_ID
  AND src.LEDGER_ID = ledger.INTEGRATION_ID
  
-- GL Account dimension (SCD Type 2)
LEFT JOIN workspace.oracle_edw.w_gl_account_d acct
  ON src.DATASOURCE_NUM_ID = acct.DATASOURCE_NUM_ID
  AND src.GL_ACCOUNT_ID = acct.INTEGRATION_ID
  AND src.BALANCE_DATE >= acct.EFFECTIVE_FROM_DT
  AND src.BALANCE_DATE < acct.EFFECTIVE_TO_DT
  AND acct.DELETE_FLG = 'N'
  
-- Natural Account dimension (SCD Type 2)
LEFT JOIN workspace.oracle_edw.w_natural_account_d nat_acct
  ON src.DATASOURCE_NUM_ID = nat_acct.DATASOURCE_NUM_ID
  AND src.NATURAL_ACCOUNT_ID = nat_acct.INTEGRATION_ID
  AND src.BALANCE_DATE >= nat_acct.EFFECTIVE_FROM_DT
  AND src.BALANCE_DATE < nat_acct.EFFECTIVE_TO_DT
  
-- Balancing Segment dimension (SCD Type 2)
LEFT JOIN workspace.oracle_edw.w_balancing_segment_d bal_seg
  ON src.DATASOURCE_NUM_ID = bal_seg.DATASOURCE_NUM_ID
  AND src.BALANCING_SEGMENT_ID = bal_seg.INTEGRATION_ID
  AND src.BALANCE_DATE >= bal_seg.EFFECTIVE_FROM_DT
  AND src.BALANCE_DATE < bal_seg.EFFECTIVE_TO_DT
  
-- Cost Center dimension (SCD Type 2)
LEFT JOIN workspace.oracle_edw.w_cost_center_d cc
  ON src.DATASOURCE_NUM_ID = cc.DATASOURCE_NUM_ID
  AND src.COST_CENTER_ID = cc.INTEGRATION_ID
  AND src.BALANCE_DATE >= cc.EFFECTIVE_FROM_DT
  AND src.BALANCE_DATE < cc.EFFECTIVE_TO_DT
  
-- Company Org dimension (SCD Type 2)
LEFT JOIN workspace.oracle_edw.w_int_org_d co_org
  ON src.DATASOURCE_NUM_ID = co_org.DATASOURCE_NUM_ID
  AND src.COMPANY_ORG_ID = co_org.INTEGRATION_ID
  AND src.BALANCE_DATE >= co_org.EFFECTIVE_FROM_DT
  AND src.BALANCE_DATE < co_org.EFFECTIVE_TO_DT
  
-- Business Area Org dimension (SCD Type 2)
LEFT JOIN workspace.oracle_edw.w_int_org_d ba_org
  ON src.DATASOURCE_NUM_ID = ba_org.DATASOURCE_NUM_ID
  AND src.BUSN_AREA_ORG_ID = ba_org.INTEGRATION_ID
  AND src.BALANCE_DATE >= ba_org.EFFECTIVE_FROM_DT
  AND src.BALANCE_DATE < ba_org.EFFECTIVE_TO_DT
  
-- Calendar dimension
LEFT JOIN workspace.oracle_edw.w_mcal_context_g mcal
  ON src.LEDGER_ID = mcal.LEDGER_ID
  AND src.DATASOURCE_NUM_ID = mcal.DATASOURCE_NUM_ID
  AND mcal.CONTEXT_CLASS = 'GL'
  
-- GL Segments (simplified - only showing 5)
LEFT JOIN workspace.oracle_edw.w_gl_segment_d seg1
  ON src.DATASOURCE_NUM_ID = seg1.DATASOURCE_NUM_ID
  AND src.GL_SEGMENT1_ID = seg1.INTEGRATION_ID
  AND seg1.CURRENT_FLG = 'Y'
  
LEFT JOIN workspace.oracle_edw.w_gl_segment_d seg2
  ON src.DATASOURCE_NUM_ID = seg2.DATASOURCE_NUM_ID
  AND src.GL_SEGMENT2_ID = seg2.INTEGRATION_ID
  AND seg2.CURRENT_FLG = 'Y'
  
LEFT JOIN workspace.oracle_edw.w_gl_segment_d seg3
  ON src.DATASOURCE_NUM_ID = seg3.DATASOURCE_NUM_ID
  AND src.GL_SEGMENT3_ID = seg3.INTEGRATION_ID
  AND seg3.CURRENT_FLG = 'Y'
  
LEFT JOIN workspace.oracle_edw.w_gl_segment_d seg4
  ON src.DATASOURCE_NUM_ID = seg4.DATASOURCE_NUM_ID
  AND src.GL_SEGMENT4_ID = seg4.INTEGRATION_ID
  AND seg4.CURRENT_FLG = 'Y'
  
LEFT JOIN workspace.oracle_edw.w_gl_segment_d seg5
  ON src.DATASOURCE_NUM_ID = seg5.DATASOURCE_NUM_ID
  AND src.GL_SEGMENT5_ID = seg5.INTEGRATION_ID
  AND seg5.CURRENT_FLG = 'Y'
  
-- Project/Program segments
LEFT JOIN workspace.oracle_edw.w_gl_segment_d proj_seg
  ON src.DATASOURCE_NUM_ID = proj_seg.DATASOURCE_NUM_ID
  AND src.PROJECT_ID = proj_seg.INTEGRATION_ID
  AND proj_seg.CURRENT_FLG = 'Y'
  
LEFT JOIN workspace.oracle_edw.w_gl_segment_d prog_seg
  ON src.DATASOURCE_NUM_ID = prog_seg.DATASOURCE_NUM_ID
  AND src.PROGRAM_ID = prog_seg.INTEGRATION_ID
  AND prog_seg.CURRENT_FLG = 'Y'
  
-- Treasury Symbol
LEFT JOIN workspace.oracle_edw.w_treasury_symbol_d treas
  ON src.DATASOURCE_NUM_ID = treas.DATASOURCE_NUM_ID
  AND CONCAT(
    src.BALANCING_SEGMENT_NUM, '~',
    src.BALANCING_SEGMENT_ATTRIB, '~',
    src.LEDGER_ID
  ) = treas.INTEGRATION_ID
  
-- Dataset Security
LEFT JOIN workspace.oracle_edw.w_gl_dataset_sec_d dataset_sec
  ON src.DATASOURCE_NUM_ID = dataset_sec.DATASOURCE_NUM_ID
  AND CONCAT(
    src.LEDGER_ID, '~',
    src.BALANCING_SEGMENT_NUM, '~',
    src.MANAGEMENT_SEGMENT_NUM
  ) = dataset_sec.INTEGRATION_ID
  
-- Balance Type
LEFT JOIN workspace.oracle_edw.w_xact_type_d balance_type
  ON src.DATASOURCE_NUM_ID = balance_type.DATASOURCE_NUM_ID
  AND src.BALANCE_TYPE_ID = balance_type.INTEGRATION_ID
  
-- Budget dimensions
LEFT JOIN workspace.oracle_edw.w_budget_d budget
  ON src.DATASOURCE_NUM_ID = budget.DATASOURCE_NUM_ID
  AND src.BUDGET_ID = budget.INTEGRATION_ID
  
LEFT JOIN workspace.oracle_edw.w_mcal_period_d budget_period
  ON src.DATASOURCE_NUM_ID = budget_period.DATASOURCE_NUM_ID
  AND src.BUDGET_PERIOD_ID = budget_period.INTEGRATION_ID
  
-- Exchange rates for global currency conversion
LEFT JOIN workspace.oracle_edw.w_global_exch_rate_g exch_rate
  ON src.LOC_CURR_CODE = exch_rate.FROM_CURR_CODE
  AND DATE_TRUNC('day', src.BALANCE_DATE) = exch_rate.RATE_DATE
  AND exch_rate.RATE_TYPE = 'Average';

In [ ]:
%sql
-- 4) CHANGE DETECTION: Determine Insert/Update/No-Action
CREATE OR REPLACE TEMP VIEW stg_balance_with_action AS
SELECT
  s.*,
  CASE
    -- No change: Record exists and all change tracking dates match
    WHEN t.INTEGRATION_ID IS NOT NULL
      AND (t.CHANGED_ON_DT = s.CHANGED_ON_DT OR (t.CHANGED_ON_DT IS NULL AND s.CHANGED_ON_DT IS NULL))
      AND (t.AUX1_CHANGED_ON_DT = s.AUX1_CHANGED_ON_DT OR (t.AUX1_CHANGED_ON_DT IS NULL AND s.AUX1_CHANGED_ON_DT IS NULL))
      AND (t.AUX2_CHANGED_ON_DT = s.AUX2_CHANGED_ON_DT OR (t.AUX2_CHANGED_ON_DT IS NULL AND s.AUX2_CHANGED_ON_DT IS NULL))
      AND (t.AUX3_CHANGED_ON_DT = s.AUX3_CHANGED_ON_DT OR (t.AUX3_CHANGED_ON_DT IS NULL AND s.AUX3_CHANGED_ON_DT IS NULL))
      AND (t.AUX4_CHANGED_ON_DT = s.AUX4_CHANGED_ON_DT OR (t.AUX4_CHANGED_ON_DT IS NULL AND s.AUX4_CHANGED_ON_DT IS NULL))
    THEN 'N'
    -- Update: Record exists but has changes
    WHEN t.INTEGRATION_ID IS NOT NULL THEN 'U'
    -- Insert: New record
    ELSE 'I'
  END AS IND_UPDATE
FROM stg_balance_with_dimensions s
LEFT JOIN workspace.oracle_edw.w_gl_balance_f t
  ON s.DATASOURCE_NUM_ID = t.DATASOURCE_NUM_ID
  AND s.INTEGRATION_ID = t.INTEGRATION_ID;

In [ ]:
%sql
-- 5) MERGE: Upsert data into target fact table
MERGE INTO workspace.oracle_edw.w_gl_balance_f AS tgt
USING (
  SELECT
    s.INTEGRATION_ID,
    s.DATASOURCE_NUM_ID,
    s.LEDGER_WID,
    s.GL_ACCOUNT_WID,
    s.NATURAL_ACCOUNT_WID,
    s.BALANCING_SEGMENT_WID,
    s.COST_CENTER_WID,
    s.COMPANY_ORG_WID,
    s.BUSN_AREA_ORG_WID,
    s.BALANCE_DT_WID,
    s.BALANCE_TM_WID,
    s.MCAL_CAL_WID,
    s.TREASURY_SYMBOL_WID,
    s.GL_DATASET_SEC_WID,
    s.GL_SEGMENT1_WID,
    s.GL_SEGMENT2_WID,
    s.GL_SEGMENT3_WID,
    s.GL_SEGMENT4_WID,
    s.GL_SEGMENT5_WID,
    s.SEG_PROJECT_WID,
    s.SEG_PROGRAM_WID,
    s.BALANCE_TYPE_WID,
    s.BUDGET_WID,
    s.BUDGET_PERIOD_WID,
    s.DB_CR_IND,
    s.BALANCE_ACCT_AMT,
    s.BALANCE_LOC_AMT,
    s.BALANCE_GLOBAL1_AMT,
    s.BALANCE_GLOBAL2_AMT,
    s.BALANCE_GLOBAL3_AMT,
    s.ACTIVITY_ACCT_AMT,
    s.ACTIVITY_LOC_AMT,
    s.ACTIVITY_GLOBAL1_AMT,
    s.ACTIVITY_GLOBAL2_AMT,
    s.ACTIVITY_GLOBAL3_AMT,
    s.CARRY_FORWARD_ACCT_AMT,
    s.CARRY_FORWARD_LOC_AMT,
    s.CARRY_FORWARD_GLOBAL1_AMT,
    s.CARRY_FORWARD_GLOBAL2_AMT,
    s.CARRY_FORWARD_GLOBAL3_AMT,
    s.ACCT_CURR_CODE,
    s.LOC_CURR_CODE,
    s.BALANCE_TYPE_FLAG,
    s.TRANSLATED_FLAG,
    s.SUMMARY_ACCOUNT_FLAG,
    s.FINANCIAL_GL_FLG,
    s.BUDGETARY_CONTROL_FLG,
    s.BALANCE_ID,
    s.CREATED_ON_DT,
    s.CHANGED_ON_DT,
    s.AUX1_CHANGED_ON_DT,
    s.AUX2_CHANGED_ON_DT,
    s.AUX3_CHANGED_ON_DT,
    s.AUX4_CHANGED_ON_DT,
    s.TENANT_ID,
    s.X_CUSTOM,
    s.IND_UPDATE,
    p.ETL_PROC_WID,
    current_timestamp() AS CURRENT_TS
  FROM stg_balance_with_action s
  CROSS JOIN etl_params p
  WHERE s.IND_UPDATE IN ('I', 'U')  -- Only process inserts and updates
) src
ON tgt.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND tgt.INTEGRATION_ID = src.INTEGRATION_ID
WHEN MATCHED AND src.IND_UPDATE = 'U' THEN
  UPDATE SET
    tgt.LEDGER_WID = src.LEDGER_WID,
    tgt.GL_ACCOUNT_WID = src.GL_ACCOUNT_WID,
    tgt.NATURAL_ACCOUNT_WID = src.NATURAL_ACCOUNT_WID,
    tgt.BALANCING_SEGMENT_WID = src.BALANCING_SEGMENT_WID,
    tgt.COST_CENTER_WID = src.COST_CENTER_WID,
    tgt.COMPANY_ORG_WID = src.COMPANY_ORG_WID,
    tgt.BUSN_AREA_ORG_WID = src.BUSN_AREA_ORG_WID,
    tgt.BALANCE_DT_WID = src.BALANCE_DT_WID,
    tgt.BALANCE_TM_WID = src.BALANCE_TM_WID,
    tgt.MCAL_CAL_WID = src.MCAL_CAL_WID,
    tgt.TREASURY_SYMBOL_WID = src.TREASURY_SYMBOL_WID,
    tgt.GL_DATASET_SEC_WID = src.GL_DATASET_SEC_WID,
    tgt.GL_SEGMENT1_WID = src.GL_SEGMENT1_WID,
    tgt.GL_SEGMENT2_WID = src.GL_SEGMENT2_WID,
    tgt.GL_SEGMENT3_WID = src.GL_SEGMENT3_WID,
    tgt.GL_SEGMENT4_WID = src.GL_SEGMENT4_WID,
    tgt.GL_SEGMENT5_WID = src.GL_SEGMENT5_WID,
    tgt.SEG_PROJECT_WID = src.SEG_PROJECT_WID,
    tgt.SEG_PROGRAM_WID = src.SEG_PROGRAM_WID,
    tgt.BALANCE_TYPE_WID = src.BALANCE_TYPE_WID,
    tgt.BUDGET_WID = src.BUDGET_WID,
    tgt.BUDGET_PERIOD_WID = src.BUDGET_PERIOD_WID,
    tgt.DB_CR_IND = src.DB_CR_IND,
    tgt.BALANCE_ACCT_AMT = src.BALANCE_ACCT_AMT,
    tgt.BALANCE_LOC_AMT = src.BALANCE_LOC_AMT,
    tgt.BALANCE_GLOBAL1_AMT = src.BALANCE_GLOBAL1_AMT,
    tgt.BALANCE_GLOBAL2_AMT = src.BALANCE_GLOBAL2_AMT,
    tgt.BALANCE_GLOBAL3_AMT = src.BALANCE_GLOBAL3_AMT,
    tgt.ACTIVITY_ACCT_AMT = src.ACTIVITY_ACCT_AMT,
    tgt.ACTIVITY_LOC_AMT = src.ACTIVITY_LOC_AMT,
    tgt.ACTIVITY_GLOBAL1_AMT = src.ACTIVITY_GLOBAL1_AMT,
    tgt.ACTIVITY_GLOBAL2_AMT = src.ACTIVITY_GLOBAL2_AMT,
    tgt.ACTIVITY_GLOBAL3_AMT = src.ACTIVITY_GLOBAL3_AMT,
    tgt.CARRY_FORWARD_ACCT_AMT = src.CARRY_FORWARD_ACCT_AMT,
    tgt.CARRY_FORWARD_LOC_AMT = src.CARRY_FORWARD_LOC_AMT,
    tgt.CARRY_FORWARD_GLOBAL1_AMT = src.CARRY_FORWARD_GLOBAL1_AMT,
    tgt.CARRY_FORWARD_GLOBAL2_AMT = src.CARRY_FORWARD_GLOBAL2_AMT,
    tgt.CARRY_FORWARD_GLOBAL3_AMT = src.CARRY_FORWARD_GLOBAL3_AMT,
    tgt.ACCT_CURR_CODE = src.ACCT_CURR_CODE,
    tgt.LOC_CURR_CODE = src.LOC_CURR_CODE,
    tgt.BALANCE_TYPE_FLAG = src.BALANCE_TYPE_FLAG,
    tgt.TRANSLATED_FLAG = src.TRANSLATED_FLAG,
    tgt.SUMMARY_ACCOUNT_FLAG = src.SUMMARY_ACCOUNT_FLAG,
    tgt.FINANCIAL_GL_FLG = src.FINANCIAL_GL_FLG,
    tgt.BUDGETARY_CONTROL_FLG = src.BUDGETARY_CONTROL_FLG,
    tgt.BALANCE_ID = src.BALANCE_ID,
    tgt.CHANGED_ON_DT = src.CHANGED_ON_DT,
    tgt.AUX1_CHANGED_ON_DT = src.AUX1_CHANGED_ON_DT,
    tgt.AUX2_CHANGED_ON_DT = src.AUX2_CHANGED_ON_DT,
    tgt.AUX3_CHANGED_ON_DT = src.AUX3_CHANGED_ON_DT,
    tgt.AUX4_CHANGED_ON_DT = src.AUX4_CHANGED_ON_DT,
    tgt.TENANT_ID = src.TENANT_ID,
    tgt.X_CUSTOM = src.X_CUSTOM,
    tgt.W_UPDATE_DT = src.CURRENT_TS,
    tgt.ETL_PROC_WID = src.ETL_PROC_WID
WHEN NOT MATCHED AND src.IND_UPDATE = 'I' THEN
  INSERT (
    INTEGRATION_ID,
    DATASOURCE_NUM_ID,
    LEDGER_WID,
    GL_ACCOUNT_WID,
    NATURAL_ACCOUNT_WID,
    BALANCING_SEGMENT_WID,
    COST_CENTER_WID,
    COMPANY_ORG_WID,
    BUSN_AREA_ORG_WID,
    BALANCE_DT_WID,
    BALANCE_TM_WID,
    MCAL_CAL_WID,
    TREASURY_SYMBOL_WID,
    GL_DATASET_SEC_WID,
    GL_SEGMENT1_WID,
    GL_SEGMENT2_WID,
    GL_SEGMENT3_WID,
    GL_SEGMENT4_WID,
    GL_SEGMENT5_WID,
    SEG_PROJECT_WID,
    SEG_PROGRAM_WID,
    BALANCE_TYPE_WID,
    BUDGET_WID,
    BUDGET_PERIOD_WID,
    DB_CR_IND,
    BALANCE_ACCT_AMT,
    BALANCE_LOC_AMT,
    BALANCE_GLOBAL1_AMT,
    BALANCE_GLOBAL2_AMT,
    BALANCE_GLOBAL3_AMT,
    ACTIVITY_ACCT_AMT,
    ACTIVITY_LOC_AMT,
    ACTIVITY_GLOBAL1_AMT,
    ACTIVITY_GLOBAL2_AMT,
    ACTIVITY_GLOBAL3_AMT,
    CARRY_FORWARD_ACCT_AMT,
    CARRY_FORWARD_LOC_AMT,
    CARRY_FORWARD_GLOBAL1_AMT,
    CARRY_FORWARD_GLOBAL2_AMT,
    CARRY_FORWARD_GLOBAL3_AMT,
    ACCT_CURR_CODE,
    LOC_CURR_CODE,
    BALANCE_TYPE_FLAG,
    TRANSLATED_FLAG,
    SUMMARY_ACCOUNT_FLAG,
    FINANCIAL_GL_FLG,
    BUDGETARY_CONTROL_FLG,
    BALANCE_ID,
    CREATED_ON_DT,
    CHANGED_ON_DT,
    AUX1_CHANGED_ON_DT,
    AUX2_CHANGED_ON_DT,
    AUX3_CHANGED_ON_DT,
    AUX4_CHANGED_ON_DT,
    TENANT_ID,
    X_CUSTOM,
    W_INSERT_DT,
    W_UPDATE_DT,
    ETL_PROC_WID,
    PROFIT_CENTER_WID
  )
  VALUES (
    src.INTEGRATION_ID,
    src.DATASOURCE_NUM_ID,
    src.LEDGER_WID,
    src.GL_ACCOUNT_WID,
    src.NATURAL_ACCOUNT_WID,
    src.BALANCING_SEGMENT_WID,
    src.COST_CENTER_WID,
    src.COMPANY_ORG_WID,
    src.BUSN_AREA_ORG_WID,
    src.BALANCE_DT_WID,
    src.BALANCE_TM_WID,
    src.MCAL_CAL_WID,
    src.TREASURY_SYMBOL_WID,
    src.GL_DATASET_SEC_WID,
    src.GL_SEGMENT1_WID,
    src.GL_SEGMENT2_WID,
    src.GL_SEGMENT3_WID,
    src.GL_SEGMENT4_WID,
    src.GL_SEGMENT5_WID,
    src.SEG_PROJECT_WID,
    src.SEG_PROGRAM_WID,
    src.BALANCE_TYPE_WID,
    src.BUDGET_WID,
    src.BUDGET_PERIOD_WID,
    src.DB_CR_IND,
    src.BALANCE_ACCT_AMT,
    src.BALANCE_LOC_AMT,
    src.BALANCE_GLOBAL1_AMT,
    src.BALANCE_GLOBAL2_AMT,
    src.BALANCE_GLOBAL3_AMT,
    src.ACTIVITY_ACCT_AMT,
    src.ACTIVITY_LOC_AMT,
    src.ACTIVITY_GLOBAL1_AMT,
    src.ACTIVITY_GLOBAL2_AMT,
    src.ACTIVITY_GLOBAL3_AMT,
    src.CARRY_FORWARD_ACCT_AMT,
    src.CARRY_FORWARD_LOC_AMT,
    src.CARRY_FORWARD_GLOBAL1_AMT,
    src.CARRY_FORWARD_GLOBAL2_AMT,
    src.CARRY_FORWARD_GLOBAL3_AMT,
    src.ACCT_CURR_CODE,
    src.LOC_CURR_CODE,
    src.BALANCE_TYPE_FLAG,
    src.TRANSLATED_FLAG,
    src.SUMMARY_ACCOUNT_FLAG,
    src.FINANCIAL_GL_FLG,
    src.BUDGETARY_CONTROL_FLG,
    src.BALANCE_ID,
    src.CREATED_ON_DT,
    src.CHANGED_ON_DT,
    src.AUX1_CHANGED_ON_DT,
    src.AUX2_CHANGED_ON_DT,
    src.AUX3_CHANGED_ON_DT,
    src.AUX4_CHANGED_ON_DT,
    src.TENANT_ID,
    src.X_CUSTOM,
    src.CURRENT_TS,
    src.CURRENT_TS,
    src.ETL_PROC_WID,
    0
  );

In [ ]:
%sql
-- 6) UPDATE CONTROL TABLE: Upsert W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT
    p.DATASOURCE_NUM_ID AS datasource_num_id,
    'SILOS_SIL_GLBALANCEFACT' AS package_name,
    'W_GL_BALANCE_F' AS target_table_name,
    p.ETL_USAGE_CODE AS etl_usage_code,
    p.ETL_PROC_WID AS etl_proc_wid,
    p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date,
    (SELECT MAX(CHANGED_ON_DT) FROM stg_balance_with_action) AS last_max_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.package_name = src.package_name
  AND tgt.etl_usage_code = src.etl_usage_code
WHEN MATCHED THEN
  UPDATE SET
    tgt.target_table_name = src.target_table_name,
    tgt.etl_proc_wid = src.etl_proc_wid,
    tgt.load_plan_id = src.load_plan_id,
    tgt.wip_load_start_date = src.wip_load_start_date,
    tgt.last_max_date = src.last_max_date,
    tgt.etl_load_date = src.etl_load_date,
    tgt.committed = src.committed
WHEN NOT MATCHED THEN
  INSERT (
    datasource_num_id, package_name, target_table_name, etl_usage_code,
    etl_proc_wid, load_plan_id, wip_load_start_date, last_max_date, etl_load_date, committed
  )
  VALUES (
    src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
    src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, src.last_max_date, src.etl_load_date, src.committed
  );

In [ ]:
%sql
-- 7) INSERT HISTORY LOG: Record ETL execution in history table
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  p.DATASOURCE_NUM_ID,
  'SILOS_SIL_GLBALANCEFACT',
  'W_GL_BALANCE_F',
  p.ETL_USAGE_CODE,
  p.ETL_PROC_WID,
  p.EXECUTION_ID,
  p.ETL_PROC_WID AS SESSION_ID,
  date_add(current_timestamp(), -p.PRUNE_DAYS) AS WIP_LOAD_START_DATE,
  (SELECT MAX(CHANGED_ON_DT) FROM stg_balance_with_action) AS LAST_MAX_DATE,
  current_timestamp() AS ETL_LOAD_DATE,
  CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS COMMITTED
FROM etl_params p;

In [ ]:
%sql
-- 8) CLEANUP: Drop temporary views
DROP VIEW IF EXISTS stg_balance_with_action;
DROP VIEW IF EXISTS stg_balance_with_dimensions;
DROP VIEW IF EXISTS stg_balance_source;
DROP VIEW IF EXISTS etl_params;